In [71]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from tetris_main import *
from utils import *
import random
from library.hill_climbing.hc import hill_climb
from library.simulated_annealing.sa import simulated_annealing
from library.genetic_algorithm import gao
import pandas as pd

In [42]:
grid_shape = (8, 8)
pieces = pieces_generator(grid_shape)
grid, pieces_coordinates = tetrimino_fitter(pieces, grid_shape)

In [63]:
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 1],
            y=pieces_coordinates[i][:, 0],
            marker_size=30,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        width=600,
        height=600,
        xaxis=dict(range=[-0.5, grid.shape[1] - 0.5], fixedrange=True, showticklabels=False),
        yaxis=dict(range=[grid.shape[0] - 0.5, -0.5], tickfont_size=12, showticklabels=False),
    ),
)
fig.write_html(f"graphs/test_{grid.size}_size.html")
# fig.write_image(f"graphs/test_{grid.size}_size.png", scale=2)
# fig.show()


In [75]:
pop_size = 10
pieces_list = pieces_generator(grid_shape, rotation=False)
# Generate each individual with hill climbing
individuals = [generate_individual(pieces_list, grid_shape, hc_hardstop=5) for i in range(pop_size)]

In [76]:
pop = Population(individuals.copy(), "max", 2, pieces_list, grid_shape)
gao.GAO(
    pop=pop,
    generations=50,
    selection_type="fps",
    tournament_size=4,
    crossover_type="cycle",
    p_crossover=0.1,
    mutation_type="inverted",
    p_mutation=0.3,
    n_mutations=5,
    p_rot_mut=0.2,
    p_adoption=0.1,
    hc_hardstop=5,
)
grid, pieces_coordinates = tetrimino_fitter(pop.elites[0].representation, grid_shape)

In [ ]:
def plot_grid(
    population: Population,
    grid,
    pieces_coordinates: list,
    save_html_name: str = None,
    save_png_name: str = None,
    marker_size: int = 50,
    width: int = 600,
    height: int = 600,
):
    fig = go.Figure(
        data=[
            go.Scatter(
                x=pieces_coordinates[i][:, 1],
                y=pieces_coordinates[i][:, 0],
                marker_size=marker_size,
                marker_symbol="square",
                mode="markers",
            )
            for i in range(len(pieces_coordinates))
        ],
        layout=dict(
            margin=dict(l=00, r=00, t=80, b=0),
            width=width,
            height=height,
            title=f"<b>Fitness:</b> {int(population.elites[0].fitness)}/{int(np.sum(100*np.ones_like(grid)))}<br><b>Pieces used:</b> {len(pieces_coordinates)}/{len(pieces_list)}",
            xaxis=dict(
                range=[-0.5, grid.shape[1] - 0.5], fixedrange=True, showticklabels=False
            ),
            yaxis=dict(
                range=[-0.5, grid.shape[0] - 0.5],
                tickfont_size=12,
                showticklabels=False,
            ),
        ),
    )
    fig.update_layout(showlegend=False)
    if save_html_name != None:
        fig.write_html(f"graphs/{save_html_name}.html")
    if save_png_name != None:
        fig.write_image(f"graphs/{save_png_name}.png", scale=2)

    fig.show()


In [74]:
plot_grid(pop, grid, pieces_coordinates, save_html_name="test")

In [69]:
fig = px.line(pop.fitness_history, title="", labels={"index": "Generation", "value": "Fitness"}, width=600, height=300)
fig.update_layout(yaxis_tickformat="0", margin=dict(l=60, r=40, t=20, b=0))
fig.show()